In [ ]:
import numpy as np
import neurokit2 as nk

import pyhrv
import pyhrv.tools as tools
import pyhrv.frequency_domain as fd

import pandas as pd
import os, json
from tsfresh import extract_features
import matplotlib.pyplot as plt

In [ ]:
workDir = r'C:\Users\kein9\OneDrive\桌面\LAB\RelaxingBC_AC'
os.chdir(workDir)

In [ ]:
feature_names = ['RRV_RMSSD', 'RRV_MeanBB', 'RRV_SDBB', 'RRV_SDSD', 'RRV_CVBB',
       'RRV_CVSD', 'RRV_MedianBB', 'RRV_MadBB', 'RRV_MCVBB', 'RRV_VLF',
       'RRV_LF', 'RRV_HF', 'RRV_LFHF', 'RRV_LFn', 'RRV_HFn', 'RRV_SD1',
       'RRV_SD2', 'RRV_SD2SD1', 'RRV_ApEn', 'RRV_SampEn', 'RRV_DFA_alpha1',
       ]
print(len(feature_names))
       
# lable_names = ['addictLabel', 'stateLabel', 'trainingCount']
lable_names = ['addictLabel', 'stateLabel']

data_name = ['data_name']

# testState = ['pre', 'VR', 'post']
# stageDuration = [300, 480, 300]
# sampleRate = 256
sampleRate = 200
# stagePoint = [-1080*sampleRate, -780*sampleRate, -300*sampleRate,]

inputPath = "DataBC&AC/BC/RES/"
rriPath = "DataBC&AC/BC/RRI/"
featureOutputPath = "FeatureBC&AC/BC/"

dataInfoOld = ['A01', 'A02', 'A03', 'A04',
               'N01', 'N02', 'N03', 'N04', 'N05', 'N06', 'N08', 'N09', 'N10']

dataInfoLack = ['A106', 'A72', 'A74','A78', 'A80', 'A82', 'A84', 'A86', 
                'A69', 'A75', 'A77', 'A81', 'A83', 'A85', 'A91', 'A99']

In [ ]:
def feature_Cal(resData, sampleRate, dataCol = "RESdata"):
    cleaned = nk.rsp_clean(resData[dataCol], sampling_rate=sampleRate)

    # Extract peaks
    _, peaks_dict = nk.rsp_peaks(cleaned)
    info = nk.rsp_fixpeaks(peaks_dict)

    rsp_rate = nk.rsp_rate(cleaned, peaks_dict, sampling_rate=sampleRate)
    rrv = nk.rsp_rrv(rsp_rate, info, sampling_rate=sampleRate, show=True)
    return rrv


In [ ]:
def set_label(dataInfo):
    addictLabel = int(dataInfo[0][0] == 'A')

    # if len(dataInfo) == 2:
    #     trainingCount = 1
    #     state = dataInfo[1]
    # elif len(dataInfo) == 3:
    #     trainingCount = int(dataInfo[1])
    #     state = dataInfo[2]

    if dataInfo[3] == "PreTest":
        state = 0
    elif dataInfo[3] == "VRTest":
        state = 1
    else:
        state = 2
    
    stateLabel = state

    # label = {'addictLabel' : addictLabel, 'stateLabel':stateLabel, 'trainingCount': trainingCount}
    label = {'addictLabel' : addictLabel, 'stateLabel':stateLabel}
    
    return label


In [ ]:
def CalRESFeatures(filePath, sampleRate, output = False):
    
    dataList = os.listdir(filePath)
    name = pd.DataFrame(columns = data_name)
    result = pd.DataFrame(columns=feature_names)
    lable = pd.DataFrame(columns=lable_names)

    for dataName in dataList:
        # if dataName == 'A09_1.csv' or dataName == 'A09_2.csv':
        #     continue

        dataInfo = dataName[:-4].split('_')
        
        data = pd.read_csv("%s%s"%(filePath, dataName), names=['TimeStamp', 'RESdata'])

        # for i in range(0, 3): #3 stages
        #     if i != 2:
        #         stageData = data[stagePoint[i]:stagePoint[i+1]]
        #     elif i == 2:
        #         stageData = data[stagePoint[i]:]

        #     features = feature_Cal(stageData, sampleRate, 'RESdata')
        #     name = name.append({"data_name" : "%s_%s"%(dataName[:-4], testState[i])}, ignore_index= True)
        #     result = result.append(features, ignore_index=True)
        #     lable = lable.append(set_label(dataInfo=dataInfo+[i]), ignore_index=True)

        # if dataInfo[0] in dataInfoOld:
        #     features = feature_Cal(data, sampleRate, 'RESdata')
        if dataInfo[0] in dataInfoLack:
            features = feature_Cal(data, 15, 'RESdata')
        else:
            features = feature_Cal(data, 20, 'RESdata')
        # name = name.append({"data_name" : "%s_%s"%(dataName[:-4], testState[i])}, ignore_index= True)
        datanameDF = pd.DataFrame({"data_name" : ["%s%s_%s"%(dataInfo[0], dataInfo[1], dataInfo[3][:-4])]})
        name = pd.concat([name, datanameDF], axis=0, ignore_index=True)
        # result = result.append(features, ignore_index=True)
        result = pd.concat([result, pd.DataFrame(features, index=[0])], axis=0, ignore_index=True)
        # lable = lable.append(set_label(dataInfo=dataInfo+[i]), ignore_index=True)
        lable = pd.concat([lable, pd.DataFrame(set_label(dataInfo), index=[0])], axis=0, ignore_index=True)
        
    result = name.join(result)
    result = result.join(lable)

        # result = result.fillna(result.mean())
    return result

In [ ]:
feature = CalRESFeatures(inputPath, sampleRate=sampleRate, output=False)

In [14]:
feature.to_csv('FeatureBC&AC/BC/RES_test.csv')
feature

,data_name,RRV_RMSSD,RRV_MeanBB,RRV_SDBB,RRV_SDSD,RRV_CVBB,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,...,RRV_MFDFA_alpha2_Width,RRV_MFDFA_alpha2_Peak,RRV_MFDFA_alpha2_Mean,RRV_MFDFA_alpha2_Max,RRV_MFDFA_alpha2_Delta,RRV_MFDFA_alpha2_Asymmetry,RRV_MFDFA_alpha2_Fluctuation,RRV_MFDFA_alpha2_Increment,addictLabel,stateLabel
0,A107VR_Post,4504.735022,7421.363636,3753.026052,4525.430512,0.505706,0.606996,6975.0,3521.175,0.504828,...,0.816007,0.656882,0.714846,0.211034,-0.239898,-0.428966,7.167800e-05,0.033848,1,2
1,A107VR_Pre,5306.368165,8500.000000,4394.392979,5350.695024,0.516987,0.624279,8600.0,5559.750,0.646483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,A10VR_Post,4879.652952,8664.957265,3740.950373,4900.822434,0.431733,0.563148,8450.0,2149.770,0.254411,...,0.509769,0.330786,0.402343,0.351083,-0.360120,-0.359627,1.576455e-05,0.011949,1,2
3,A10VR_Pre,3556.880324,8287.109375,2567.369791,3570.962112,0.309803,0.429206,8200.0,1927.380,0.235046,...,0.262962,0.553286,0.549314,0.714547,0.022981,-0.515106,7.895340e-08,0.002303,1,0
4,A111VR_Post,5300.039974,8139.075630,4188.736712,5322.450394,0.514645,0.651185,7400.0,3632.370,0.490861,...,0.441064,0.557159,0.565593,0.515296,-0.050677,-0.480878,1.218143e-06,0.007174,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,A6VR_Pre,4485.961007,7045.762712,4043.643192,4503.894572,0.573911,0.636689,6000.0,3187.590,0.531265,...,0.652105,0.580555,0.685890,0.102260,-0.526421,-0.338470,2.104971e-05,0.018653,1,0
74,A7VR_Post,4255.125908,7267.391304,3632.094048,4273.864385,0.499780,0.585509,6050.0,1779.120,0.294069,...,0.931751,0.632401,0.638828,0.142727,-0.022903,-0.493102,3.754591e-05,0.038175,1,2
75,A7VR_Pre,4485.934157,7658.333333,3879.437471,4509.321874,0.506564,0.585759,6175.0,3113.460,0.504204,...,0.702754,0.619241,0.827314,-0.058983,-0.893571,-0.203919,1.278859e-04,0.030224,1,0
76,A9VR_Post,3821.096701,7125.000000,2734.657200,3836.460272,0.383812,0.536294,6900.0,1927.380,0.279330,...,0.390663,0.660760,0.738318,0.408910,-0.402400,-0.301470,8.008094e-06,0.006742,1,2


In [16]:
res_data = pd.read_csv("FeatureBC&AC/BC/RES_test.csv", header=0, index_col=0).drop(['RRV_SampEn', 'RRV_DFA_alpha1'], axis=1)
res_data.to_csv('FeatureBC&AC/BC/RES_dc_test.csv')